In [1]:
# importing dependencies
from Bio import SeqIO
import pandas as pd
from collections import defaultdict


In [2]:
# functions


def import_fasta(fasta):

    with open(fasta) as f:
        records = SeqIO.parse(f, "fasta")
        transcripts = [dna_to_mrna(str(rec.seq)) for rec in records]
        print("sequence loaded")
        return "".join(transcripts)


def complement(string):

    result = ""
    for nuc in string:
        if nuc == "A":
            result += "U"
        elif nuc == "C":
            result += "G"
        elif nuc == "G":
            result += "C"
        elif nuc in ["U", "T"]:
            result += "A"
    return result


def uracil_to_thymine(string):
    return "".join("T" if nuc == "U" else nuc for nuc in string)


def mirna_to_mrna(string):
    return complement(string[::-1])


def dna_to_mrna(string):
    return "".join("U" if nuc == "T" else nuc for nuc in string)


def sliding_window(sequence, win_size):
    for i in range(len(sequence) - win_size + 1):
        yield sequence[i : i + win_size]


def create_mirna_matrix(fasta):
    with open(fasta) as f:

        # initializing lists to zip into a df
        names = []
        sequences = []

        for record in SeqIO.parse(f, "fasta"):

            name = str(record.id)
            if name.startswith("hsa"):  # drops all non-human entries
                sequence = str(record.seq)

                # adding name
                names.append(name)

                # adding mRNA sequence
                sequences.append(mirna_to_mrna(sequence))

    return [names, sequences]


def find_matches(sequence, mirna_matrix):  # sourcery skip: remove-redundant-if

    # mirna sequences are already converted to reverse complement (mRNA)
    mirna_names = mirna_matrix[0]
    mirna_sequences = mirna_matrix[1]

    mirna_seeds = [seq[-7:-1] for seq in mirna_sequences]

    seed_dict = defaultdict(list)

    for i in range(len(mirna_names)):
        seed_dict[mirna_seeds[i]].append((mirna_names[i], mirna_sequences[i]))

    # generates k=10 length chunks of mRNA
    generator = sliding_window(dna_to_mrna(sequence), 10)

    names = []
    coordinates = []
    seed_matches = []
    chunks = []
    match_types = []

    # for loop
    for c, chunk in enumerate(generator, start=1):

        seed = chunk[2:8]

        # if there's a seed match found
        if seed in seed_dict:
            for record in seed_dict[seed]:

                mirna_sequence = seed_dict[seed][0][1]

                if chunk[8] == "A" and chunk[1] == mirna_sequence[-8]:
                    match_types.append("8mer")
                    coordinates.append(f"{str(c + 1)}-{str(c + 8)}")
                    seed_matches.append(chunk[1:9])

                elif chunk[8] == "A":
                    match_types.append("7mer-A1")
                    coordinates.append(f"{str(c + 2)}-{str(c + 8)}")
                    seed_matches.append(chunk[2:9])

                elif chunk[1] == mirna_sequence[-8]:
                    match_types.append("7mer-m8")
                    coordinates.append(f"{str(c + 1)}-{str(c + 7)}")
                    seed_matches.append(chunk[1:8])

                else:
                    match_types.append("6mer")
                    coordinates.append(f"{str(c + 2)}-{str(c + 7)}")
                    seed_matches.append(chunk[2:8])

                chunks.append(chunk)
                names.append(record[0])

    return pd.DataFrame(
        list(
            zip(
                names,
                coordinates,
                seed_matches,
                chunks,
                match_types,
            )
        ),
        columns=[
            "name",
            "coordinates",
            "seed match",
            "chunk",
            "match_type",
        ],
    )

# running the script

test case: 3' UTR of human HMGA2_ENST00000403681_7 transcript

This gene is the example gene used in the TargetScan website, therefore I picked it.

TargetScan doesn't have the results for **HMGA2_ENST00000403681_7**, instead, it has results for **HMGA2_ENST00000403681_2**. I couldn't access the version .2 of that transcript, so I went with .7 version. I don't know if these transcripts are exactly the same or have some differences.

**question: How can I access that?**

In [3]:
# run script
fasta = "../sequences/HMGA2_ENST00000403681_7_sequence.fa"
mirna_fasta = "../sequences/mirna/mature_mirnas.fa"

s = import_fasta(fasta)
m = create_mirna_matrix(mirna_fasta)

df = find_matches(s, m)

display(df)


sequence loaded


,name,coordinates,seed match,chunk,match_type
0,hsa-miR-6789-3p,2-9,GGGCGCCA,GGGGCGCCAA,8mer
1,hsa-miR-1306-3p,6-12,GCCAACG,CGCCAACGUU,7mer-m8
2,hsa-miR-5680,17-23,AUUUCUA,CGAUUUCUAC,7mer-A1
3,hsa-miR-379-5p,20-25,UCUACC,UUUCUACCUC,6mer
4,hsa-miR-4451,20-25,UCUACC,UUUCUACCUC,6mer
...,...,...,...,...,...
2302,hsa-miR-556-3p,2976-2982,GUAAUAA,GAGUAAUAAA,7mer-A1
2303,hsa-miR-19a-5p,2991-2997,CAAAACA,GUCAAAACAG,7mer-A1
2304,hsa-miR-19b-1-5p,2991-2997,CAAAACA,GUCAAAACAG,7mer-A1
2305,hsa-miR-19b-2-5p,2991-2997,CAAAACA,GUCAAAACAG,7mer-A1


In [4]:
# results test to see how many of targetscan results are missing in our results

df.to_excel("results.xlsx")


targetscan = "TargetScan_8.0_ENST00000403681.2_predicted_targeting_details.xlsx"
ours = "results.xlsx"

ts = pd.read_excel(targetscan)
us = pd.read_excel(ours)

ts_data = [ts["miRNA"].to_list(), ts["Position in the UTR"].to_list()]
us_data = [us["name"].to_list(), us["coordinates"].to_list()]


results = [[], []]
missed_mirnas = [[], []]

for record in ts_data[1]:

    index_of_record = ts_data[1].index(record)

    if record in us_data[1]:
        results[0].append(ts_data[0][index_of_record])
        results[1].append(record)

    else:
        missed_mirnas[0].append(ts_data[0][index_of_record])
        missed_mirnas[1].append(record)


print(len(missed_mirnas[0]))

217


217 results of TargetScan are missing in our results

In [5]:
missing_but_existing = []
missing_and_not_existing = []

for record in missed_mirnas[0]:
    if record in m[0]:
        missing_but_existing.append(record)

    else:
        missing_and_not_existing.append(record)


print(len(missing_but_existing))


200


200 our of 217 missed records are exists in our miRNA database.

17 of the 217 missing are miRNAs that doesn't exist in our miRNA database.

# cells below are used for debugging, not a part of the script

In [6]:
# somewhat working


def find_matches_debug(sequence, mirna_matrix):  # sourcery skip: remove-redundant-if

    # mirna sequences are already converted to reverse complement (mRNA)
    mirna_names = mirna_matrix[0]
    mirna_sequences = mirna_matrix[1]

    mirna_seeds = [seq[-7:-1] for seq in mirna_sequences]

    seed_dict = defaultdict(list)

    for i in range(len(mirna_names)):
        seed_dict[mirna_seeds[i]].append((mirna_names[i], mirna_sequences[i]))

    # generates k=10 length chunks of mRNA
    generator = sliding_window(dna_to_mrna(sequence), 10)

    names = []

    m8_matches = []
    seeds = []
    a1s = []
    chunks = []
    match_types = []
    mirnas_with_same_seed = []

    # for loop
    for c, chunk in enumerate(generator, start=1):

        seed = chunk[2:8]

        # if there's a seed match found
        if seed in seed_dict:
            for record in seed_dict[seed]:

                mirna_sequence = seed_dict[seed][0][1]

                if chunk[8] == "A" and chunk[1] == mirna_sequence[-8]:
                    m8_matches.append(chunk[1])
                    match_types.append("8mer")

                elif chunk[8] == "A":
                    m8_matches.append("no")
                    match_types.append("7mer-A1")

                elif chunk[1] == mirna_sequence[-8]:
                    m8_matches.append(chunk[1])
                    match_types.append("7mer-m8")

                else:
                    m8_matches.append("no")
                    match_types.append("6mer")
                mirnas_with_same_seed.append(len(seed_dict[seed]))

                chunks.append(chunk)
                a1s.append(chunk[8])

                seeds.append(chunk[2:8])
                names.append(record[0])

    return (
        pd.DataFrame(
            list(
                zip(
                    names,
                    m8_matches,
                    seeds,
                    a1s,
                    chunks,
                    match_types,
                    mirnas_with_same_seed,
                )
            ),
            columns=[
                "name",
                "m8",
                "seed",
                "chunk[8]",
                "chunk",
                "match_type",
                "mirnas with same seed",
            ],
        ),
        seed_dict,
    )


In [7]:
# debug run
fasta = "../sequences/HMGA2_ENST00000403681_7_sequence.fa"
mirna_fasta = "../sequences/mirna/mature_mirnas.fa"

s = import_fasta(fasta)
m = create_mirna_matrix(mirna_fasta)

df, dict = find_matches_debug(s, m)

display(df)


sequence loaded


,name,m8,seed,chunk[8],chunk,match_type,mirnas with same seed
0,hsa-miR-6789-3p,G,GGCGCC,A,GGGGCGCCAA,8mer,1
1,hsa-miR-1306-3p,G,CCAACG,U,CGCCAACGUU,7mer-m8,1
2,hsa-miR-5680,no,AUUUCU,A,CGAUUUCUAC,7mer-A1,1
3,hsa-miR-379-5p,no,UCUACC,U,UUUCUACCUC,6mer,4
4,hsa-miR-4451,no,UCUACC,U,UUUCUACCUC,6mer,4
...,...,...,...,...,...,...,...
2302,hsa-miR-556-3p,no,GUAAUA,A,GAGUAAUAAA,7mer-A1,1
2303,hsa-miR-19a-5p,no,CAAAAC,A,GUCAAAACAG,7mer-A1,4
2304,hsa-miR-19b-1-5p,no,CAAAAC,A,GUCAAAACAG,7mer-A1,4
2305,hsa-miR-19b-2-5p,no,CAAAAC,A,GUCAAAACAG,7mer-A1,4
